Actomyosin autoencoder code

Liya Ding

2024.03

In [ ]:
import tensorflow as tf
from utils import actomyosin_data, plot64
from model.autoencoder_64 import AE, VAE, CVAE
from train_utils.autoencoder import AETrain, VAETrain, CVAETrain
import time
import argparse
from datetime import datetime
from packaging import version
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import tifffile
import os
import warnings
from itertools import cycle, islice

from skimage.transform import resize
from skimage.io import imsave, imread

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler

In [ ]:
# args
ae_type = 'AE'
net_type = 'simple'
latent_dim = 2
num_epochs = 200
learn_rate = 0.001
train_buf = 12000
batch_size = 1000

In [ ]:
data_dir = '/net/projects/CLS/actomyosin_dynamics/data/LifeAct-NMY2-GFP_patchsize_64_p95_all/NMY2_wt'
patch_size = 64

filenames = [x for x in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, x)) and ('.tif' in x)]
filenames.sort()

num_of_samples = min(len(filenames),train_buf)

train_images = np.zeros([num_of_samples, patch_size, patch_size])
train_labels= np.zeros([num_of_samples])

for filename_ind in range(num_of_samples):
    filename = filenames[filename_ind] 
             
    train_img = tifffile.imread(os.path.join(data_dir,filename))
    small_train_img = train_img/((train_img.max()))
    train_images[filename_ind,:,:] = small_train_img
    train_labels[filename_ind] = int(filename[1:5]+filename[6:10]+filename[11:15])


train_images = train_images.reshape(train_images.shape[0], 64, 64, 1).astype('float32')

In [ ]:
TRAIN_BUF = train_buf
BATCH_SIZE = batch_size

train_dataset_image = tf.data.Dataset.from_tensor_slices(train_images).batch(BATCH_SIZE)
train_dataset_label = tf.data.Dataset.from_tensor_slices(train_labels).batch(BATCH_SIZE)
train_dataset = tf.data.Dataset.zip((train_dataset_image, train_dataset_label)).shuffle(TRAIN_BUF)

In [ ]:

model = AE(latent_dim, net_type=net_type)
model_ID = ae_type+'_ld'+str(latent_dim)+'_nt'+net_type+'_bs'+ str(batch_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learn_rate)

for epoch in range(1, num_epochs + 1):
    t = time.time()
    last_loss = 0
    for train_x, _ in train_dataset:
        gradients, loss = AETrain.compute_gradients(model, train_x)
        AETrain.apply_gradients(optimizer, gradients, model.trainable_variables)
        last_loss = loss
    if epoch % 10 == 0:
        print('Epoch {}, Loss: {}, Remaining Time at This Epoch: {:.2f}'.format(
            epoch, last_loss, time.time() - t
        ))


In [ ]:
n = 10

fig_dist = plt.figure(figsize=(8, 8))
ax_dist = fig_dist.add_subplot(111)

flag_sample = 1

for x_input, y_input in train_dataset:
    if flag_sample == 1:        
        x_input_sample, y_input_sample = map(lambda x: x[:n], (x_input, y_input))
        z = model.encode(x_input_sample).numpy()

        fig1, axarr1 = plt.subplots(2, n, figsize=(n, 2))
        x_input_sample = x_input_sample.numpy().reshape([n, 64, 64])
        x_output = model.decode(z).numpy().reshape([n, 64, 64])

        for i in range(n):
            axarr1[0, i].axis('off')
            axarr1[1, i].axis('off')
            axarr1[0, i].imshow(x_input_sample[i],cmap=plt.cm.gray)
            axarr1[1, i].imshow(x_output[i],cmap=plt.cm.gray)

        fig1.savefig("results/"+model_ID+"_reconstruction.png")
        
        z = model.encode(x_input)
        Z_array = z.numpy()
        Label_array = y_input
        flag_sample = 0
    else:
        z = model.encode(x_input)
        Z_array = np.concatenate((Z_array,z.numpy()), axis=0)
        Label_array = np.concatenate((Label_array,y_input.numpy()), axis=0)
        
    
ax_dist.scatter(Z_array[:,0], Z_array[:,1], color='blue',s=0.5)
    
fig_dist.savefig("results/"+model_ID+"_distribution.png")

